<!--
tags: ["group chat", "orchestration", "RAG"]
description: |
    Implement and manage a multi-agent chat system using AutoGen, where AI assistants retrieve information, generate code, and interact collaboratively to solve complex tasks, especially in areas not covered by their training data.
-->

# Group Chat with Retrieval Augmented Generation

AutoGen supports conversable agents powered by LLMs, tools, or humans, performing tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation.

In [ ]:
%pip install autogen
%pip install pyautogen
%pip install pyautogen[autobuild]
%pip install pyautogen[retrievechat]

In [ ]:
import chromadb
import autogen

from typing_extensions import Annotated

from autogen import AssistantAgent, GroupChat, GroupChatManager, UserProxyAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.agent_builder import AgentBuilder

CACHE_SEED = 98

config_file_or_env = "./OAI_CONFIG_LIST.json"
config_list = autogen.config_list_from_json(config_file_or_env, filter_dict={"model": ["gpt-4-1106-preview", "gpt-3.5-turbo"]})

llm_config = {
    "timeout": 60,
    "cache_seed": CACHE_SEED,
    "config_list": config_list,
    "temperature": 0,
}

def termination_msg(x):
    return isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

# RetrieveUserProxyAgent is a user proxy agent that can retrieve content from a database.
autogen.agentchat.contrib.retrieve_user_proxy_agent.PROMPT_QA = """
Here is the research summary for the provided company app information:

{input_context}
"""

ceo = UserProxyAgent(
    name="CEO",
    is_termination_msg=termination_msg,
    system_message="CEO of the company. Gives tasks to the team, and expects them to be completed.",
    human_input_mode="ALWAYS",
    code_execution_config=False,
    llm_config=llm_config,
)

ceo_aid = RetrieveUserProxyAgent(
    name="Researcher",
    is_termination_msg=termination_msg,
    system_message="Assistant who has extra content retrieval power for finding company app information.",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=2,
    llm_config=llm_config,
    retrieve_config={
        "task": "qa",
        "docs_path": "https://gist.githubusercontent.com/forsbergplustwo/835f5134190902f7cca3d943b3a3b9db/raw/5c3a6460cfbe4f4e890ca40228db115aace3242c/digital_downloads_pro.txt",
        "chunk_token_size": 1000,
        "model": config_list[0]["model"],
        "client": chromadb.PersistentClient(path="/tmp/chromadb"),
        "collection_name": "groupchat",
        "get_or_create": True,
    },
    code_execution_config=False,  # we don't want to execute code in this case.
)

ceo_researcher = RetrieveAssistantAgent(
    name="researcher",
    system_message="You are a helpful research assistant. Use the [retrieve_company_app_info] function to get company and app information for the user task. Reply with the full text and initial task description.",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=2,
    llm_config=llm_config,
)

# Load agent team agents from a config file.
builder = AgentBuilder(config_file_or_env=config_file_or_env)
agent_list, _agent_configs = builder.load(
    "./marketing_agents.json",
)

def _reset_agents():
    ceo.reset()
    ceo_aid.reset()
    ceo_researcher.reset()
    # Actual agents.
    for agent in agent_list:
        agent.reset()


def call_rag_chat(message=""):
    _reset_agents()

    # Register the function for the CEO assistant agents.
    @ceo_aid.register_for_execution()
    @ceo_researcher.register_for_llm(description="Retreive company and app information.")
    def retrieve_company_app_info(
        message: Annotated[str, "The original task description."],
        app_name: Annotated[str, "App name to lookup"] = "",
        n_results: Annotated[int, "Number of results to retrieve."] = 1,
    ) -> str:
        ceo_aid.n_results = n_results
        update_context_case1, update_context_case2 = ceo_aid._check_update_context(message)
        if (update_context_case1 or update_context_case2) and ceo_aid.update_context:
            ceo_aid.problem = message if not hasattr(ceo_aid, "problem") else ceo_aid.problem
            _, ret_msg = ceo_aid._generate_retrieve_user_reply(message)
        else:
            ret_msg = ceo_aid.generate_init_message(message, n_results=n_results)
        return ret_msg if ret_msg else message


    for agent in [ceo_researcher, *agent_list]:
        agent.llm_config = llm_config
        agent.human_input_mode = "NEVER"
        agent.is_termination_msg = termination_msg

    groupchat = autogen.GroupChat(
        agents=[ceo, ceo_researcher, ceo_aid, *agent_list],
        messages=[],
        max_round=12,
        speaker_selection_method="auto",
        allow_repeat_speaker=True,
    )

    manager_llm_config = llm_config.copy()
    # manager_llm_config.pop("functions")
    manager = autogen.GroupChatManager(
        groupchat=groupchat,
        llm_config=manager_llm_config,

    )

    # Start chatting with the boss as this is the user proxy agent.
    ceo.initiate_chat(
        manager
    )

## Start Chat

In [ ]:
# Create a blog post for a new feature for our Digital Downloads Pro app. The new feature is: Set expiration dates on License Keys.
call_rag_chat(message="")

# rag_chat()
# type exit to terminate the chat